In [29]:
!pip install -q matplotlib pandas

In [34]:
import os
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files

In [28]:
# 1. Upload dos arquivos CSV
uploaded = files.upload()

Saving dados (1).csv to dados (1).csv
Saving dados (2).csv to dados (2).csv


In [32]:
# 2. Criar pastas
pasta_imagens = "/content/imagens_emg"
pasta_filtrados = "/content/csv_filtrados"
pasta_saida_zip = "/content/saida_zip"

os.makedirs(pasta_imagens, exist_ok=True)
os.makedirs(pasta_filtrados, exist_ok=True)
os.makedirs(pasta_saida_zip, exist_ok=True)

In [35]:
# 3. Função para aplicar filtro e salvar CSV + imagem
def aplicar_filtro_e_salvar(nome_arquivo, janela=5):
    # Carrega o CSV
    df = pd.read_csv(nome_arquivo)
    df.columns = ['tempo', 'sinal1', 'sinal2']

    # Aplica filtro de média móvel
    df['sinal1_filtrado'] = df['sinal1'].rolling(window=janela, min_periods=1).mean()
    df['sinal2_filtrado'] = df['sinal2'].rolling(window=janela, min_periods=1).mean()

    # Salva CSV filtrado
    nome_base = os.path.splitext(os.path.basename(nome_arquivo))[0]
    caminho_csv_filtrado = os.path.join(pasta_filtrados, f"{nome_base}_filtrado.csv")
    df.to_csv(caminho_csv_filtrado, index=False)

    # Gera e salva gráfico
    plt.figure(figsize=(10, 6))

    # Sinal 1
    plt.subplot(2, 1, 1)
    plt.plot(df['tempo'], df['sinal1'], label='Original', alpha=0.4)
    plt.plot(df['tempo'], df['sinal1_filtrado'], label='Filtrado', color='blue')
    plt.title('Sinal 1')
    plt.ylabel('Amplitude')
    plt.legend()

    # Sinal 2
    plt.subplot(2, 1, 2)
    plt.plot(df['tempo'], df['sinal2'], label='Original', alpha=0.4)
    plt.plot(df['tempo'], df['sinal2_filtrado'], label='Filtrado', color='green')
    plt.title('Sinal 2')
    plt.xlabel('Tempo (ms)')
    plt.ylabel('Amplitude')
    plt.legend()

    plt.tight_layout()

    # Salvar imagem PNG
    caminho_imagem = os.path.join(pasta_imagens, f"{nome_base}_grafico.png")
    plt.savefig(caminho_imagem)
    plt.close()

# 4. Aplicar para todos os arquivos enviados
for nome_arquivo in uploaded.keys():
    aplicar_filtro_e_salvar(nome_arquivo)

# 5. Compactar todos os arquivos gerados em um único ZIP
caminho_zip = os.path.join(pasta_saida_zip, "resultados_emg.zip")
with zipfile.ZipFile(caminho_zip, "w") as zipf:
    for pasta in [pasta_filtrados, pasta_imagens]:
        for raiz, _, arquivos in os.walk(pasta):
            for nome_arquivo in arquivos:
                caminho_completo = os.path.join(raiz, nome_arquivo)
                caminho_relativo = os.path.relpath(caminho_completo, start="/content")
                zipf.write(caminho_completo, arcname=caminho_relativo)

# 6. Download do ZIP
files.download(caminho_zip)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>